<a href="https://colab.research.google.com/github/tkd8973/ML_Project/blob/main/SW/MP_2_KNN_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf


Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/n

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
dosi=dict()

#@title 한글 폰트를 나눔폰트로 설정
plt.rc('font', family='NanumBarunGothic')

In [30]:
Jan_data = pd.read_csv('https://github.com/jaiwon880/ML_Projiect/raw/main/Data/%EC%95%84%ED%8C%8C%ED%8A%B8_%EB%A7%A4%EB%A7%A4__%EC%8B%A4%EA%B1%B0%EB%9E%98%EA%B0%80_20230321134107_1%EC%9B%94.csv')
Feb_Data = pd.read_csv('https://raw.githubusercontent.com/jaiwon880/ML_Projiect/main/Data/%EC%95%84%ED%8C%8C%ED%8A%B8_%EB%A7%A4%EB%A7%A4__%EC%8B%A4%EA%B1%B0%EB%9E%98%EA%B0%80_20230321134120_2%EC%9B%94.csv')


In [5]:
len(Jan_data['시군구'].unique())

2407

In [6]:
import seaborn as sns

In [2]:
# sns.pointplot(x='전용면적(㎡)',y='거래금액(만원)',hue='시군구',data=Jan_data)

NameError: ignored

In [ ]:
aa=Jan_data['계약년월'].astype(str)
bb=Jan_data['계약일'].astype(str)

Jan_data['date'] = aa+bb
Jan_data['date'] = pd.to_datetime(Jan_data['date'], format='%Y%m%d')
Jan_data
Jan_data.drop(['계약년월','계약일'],axis=1,inplace=True)

In [ ]:
Jan_data

In [7]:
def preprocessing(data):
    cnt=0
    global dosi
    data['거래금액(만원)'].str.strip().apply(lambda x: x.replace(',',''))
    data['거래금액(만원)']=(data['거래금액(만원)'].str.replace(',','')).str.strip().astype(int)
    data['시군구']= data['시군구'].apply(lambda x: x.split()[0])
    data.groupby(['시군구'])['거래금액(만원)'].mean()
    data.dropna(inplace=True)
    data['거래유형'] = data['거래유형'].apply(lambda x:0 if x=='중개거래' else 1)
    data['계약년월'] = data['계약년월']-202300
    for i in data['시군구'].unique():
        dosi[i] = cnt
        cnt+=1
    data['시군구'] = data['시군구'].map(dosi)
    
    return data

In [8]:
Jan = preprocessing(Jan_data)
Feb = preprocessing(Feb_Data)

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

In [11]:
train = Jan
test = Feb
X_train = train.drop('거래금액(만원)',axis=1)
y_train = train['거래금액(만원)']
X_test = test.drop('거래금액(만원)',axis=1)
y_test = test['거래금액(만원)']

In [18]:
from sklearn.metrics import mean_squared_error
models = []
for i in range(0,50,5):
    if i==0:
        continue
    model = KNeighborsRegressor(n_neighbors=i,weights='distance')
    model.fit(X_train,y_train)

    pred=model.predict(X_test)
    rmse = mean_squared_error(y_test,pred)**0.5
    
    models.append(rmse)

In [13]:
models

[23350.30197416621,
 23350.30197416621,
 23350.30197416621,
 23350.30197416621,
 23350.30197416621,
 23350.30197416621,
 23350.30197416621,
 23350.30197416621,
 23350.30197416621]

In [20]:
model = KNeighborsRegressor(n_neighbors=5,weights='distance')
model.fit(X_train,y_train)

pred=model.predict(X_test)
rmse = mean_squared_error(y_test,pred)**0.5
rmse

pred,rmse

(array([25601.32632007, 37094.04558426, 37094.04558426, ...,
        21767.04961707, 36822.99318233, 64202.15539923]), 23350.30197416621)

In [21]:
import lightgbm as lgb

model = lgb.LGBMRegressor(num_leaves=16, max_depth=4, learning_rate=0.1)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [22]:
from sklearn.metrics import r2_score, mean_squared_error

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

In [23]:
r2,mse,rmse

(0.6964648045531482, 326502865.5485267, 18069.390292661417)